# Démonstration complète SOST - Formule Equilibrium (E)
import numpy as np
import matplotlib.pyplot as plt

# 1. Implémentation
class EquilibriumDetector:
    def __init__(self, window_size=50, q=0.1, lambda_=3.0):
        self.window_size, self.q, self.lambda_ = window_size, q, lambda_
    
    def detect(self, ref_data, obs_data):
        # Mesures simplifiées
        def var(data): return np.var(data, axis=0)
        def autocorr(data): 
            return np.array([np.corrcoef(data[:-1,i], data[1:,i])[0,1] 
                           for i in range(data.shape[1])])
        
        ref_var, obs_var = var(ref_data), var(obs_data)
        ref_ac, obs_ac = autocorr(ref_data), autocorr(obs_data)
        
        # Scores
        scores_var = (obs_var - np.median(ref_var)) / (np.median(np.abs(ref_var - np.median(ref_var))) + 1e-8)
        scores_ac = (obs_ac - np.median(ref_ac)) / (np.median(np.abs(ref_ac - np.median(ref_ac))) + 1e-8)
        
        component_scores = np.minimum(scores_var, scores_ac)
        Dt = np.percentile(component_scores, self.q * 100)
        theta = np.median(component_scores) + self.lambda_ * np.median(np.abs(component_scores - np.median(component_scores)))
        
        return Dt, theta, Dt < theta

# 2. Données synthétiques
np.random.seed(42)
ref_data = np.random.randn(200, 5) * 0.5
obs_data = np.zeros((300, 5))
obs_data[:150] = np.random.randn(150, 5) * 0.5
obs_data[150:, 0] = np.random.randn(150) * 2.0  # Transition

# 3. Détection
detector = EquilibriumDetector()
Dt_scores, compatible = [], []
for t in range(50, 300):
    Dt, theta, comp = detector.detect(ref_data, obs_data[t-50:t])
    Dt_scores.append(Dt)
    compatible.append(comp)

# 4. Visualisation
plt.figure(figsize=(12, 4))
plt.plot(range(50, 300), Dt_scores, 'b-', label='Dt')
plt.axhline(y=3.0, color='r', linestyle='--', label='Seuil θ')
plt.axvline(x=150, color='gray', linestyle=':', label='Transition')
plt.legend()
plt.title("SOST - Formule Equilibrium (E)")
plt.show()

print("Démonstration terminée. Le cadre observe sans expliquer.")